In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.compat.v1.disable_eager_execution()

In [ ]:
#=============  mnist 数据集加载 =============

In [2]:

mnist = tf.keras.datasets.mnist

path = '/Users/bytedance/LearningProjects/practice_shouqianba/mnist.npz'
(x_train, y_train), (x_test, y_test) = mnist.load_data(path)
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
#=============  全keras 示例 =============

In [10]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 42us/sample - loss: 0.2931 - accuracy: 0.9157
Epoch 2/5
60000/60000 [==============================] - 2s 33us/sample - loss: 0.1427 - accuracy: 0.9577
Epoch 3/5
60000/60000 [==============================] - 2s 32us/sample - loss: 0.1051 - accuracy: 0.9680
Epoch 4/5
60000/60000 [==============================] - 2s 32us/sample - loss: 0.0879 - accuracy: 0.9729
Epoch 5/5
60000/60000 [==============================] - 2s 34us/sample - loss: 0.0743 - accuracy: 0.9770
10000/1 - 0s - loss: 0.0362 - accuracy: 0.9784


[0.07165266907969489, 0.9784]

In [16]:
type(x_train)
print(x_train.shape, y_train.shape, x_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28)


In [ ]:
#=============  tf2.0 + keras 灵活组合示例 =============

In [3]:
img = tf.compat.v1.placeholder(tf.float32, shape=(None, 28, 28))
labels = tf.compat.v1.placeholder(tf.float32, shape=(None, 10))


In [4]:
x = tf.keras.layers.Flatten(input_shape=(28, 28))(img)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
prediction = tf.keras.layers.Dense(10, activation='softmax')(x)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction, labels=labels))

train_optim = tf.compat.v1.train.AdamOptimizer().minimize(loss)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
classes = y_train.max() - y_train.min() + 1
y_train_tf = np.eye(classes)[y_train]
y_test_tf = np.eye(classes)[y_test]

print(y_train_tf[0:2])
print(x_train[0:1].shape)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(1, 28, 28)


In [6]:
with tf.compat.v1.Session() as sess:
    init = tf.compat.v1.global_variables_initializer()
    sess.run(init)
    batch_size = 60
    
    for _ in range(1000):
        idx_start, idx_end = _*batch_size, (_+1)*batch_size
        batch_x = x_train[idx_start: idx_end]
        batch_y = y_train_tf[idx_start: idx_end]
        sess.run(
            train_optim,
            feed_dict={img: batch_x, labels: batch_y}
        )
    
    acc_pred = tf.keras.metrics.categorical_accuracy(labels, prediction)
    pred = sess.run(acc_pred, feed_dict={
        img: x_test, labels: y_test_tf
    })
    print('accuracy: %.3f' % (sum(pred)/len(y_test_tf)))

accuracy: 0.939


In [20]:
batch_y

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 

In [ ]:
#=============  tf1.0 + keras 灵活组合示例 =============

In [ ]:
img = tf.placeholder(tf.float32, shape=(None, 784))
labels = tf.placeholder(tf.float32, shape=(None, 10))


In [ ]:
with tf.compat.v1.Session() as sess:
    init = tf.compat.v1.global_variables_initializer()
    sess.run(init)
    batch_size = 60
    
    for _ in range(1000):
        idx_start, idx_end = _*batch_size, (_+1)*batch_size
        batch_x = x_train[idx_start: idx_end]
        batch_y = y_train_tf[idx_start: idx_end]
        sess.run(
            train_optim,
            feed_dict={img: batch_x, labels: batch_y}
        )
    
    acc_pred = tf.keras.metrics.categorical_accuracy(labels, prediction)
    pred = sess.run(acc_pred, feed_dict={
        img: x_test, labels: y_test_tf
    })
    print('accuracy: %.3f' % (sum(pred)/len(y_test_tf)))